In [1]:
import pandas as pd
from itertools import cycle
from IPython.display import clear_output
import os
path_output = 'data/results/all.gff'
path_blast = 'data/results/'
pd.set_option('display.max_colwidth', -1)


In [2]:
#TEs
params = {}
params['LTR'] = {'file':'blast_clustered_ltr.csv.filtered'}
params['LINE'] = {'file':'blast_clustered_line.csv.filtered'}
params['SINE'] = {'file':'blast_clustered_sine.csv.filtered'}
params['TIR'] = {'file':'blast_clustered_tir.csv.filtered'}
params['MITE'] = {'file':'blast_clustered_mite.csv.filtered'}
#params['LARD'] = {'file':'blast_clustered_lard.csv.filtered'}
params['helitron'] = {'file':'blast_clustered_helitron.csv.filtered'}
params['TRIM'] = {'file':'blast_clustered_trim.csv.filtered'}


In [3]:
#read blast output
dfs = []
for k,v in params.items():
    df = pd.read_csv(path_blast + v['file'], sep='\t')
    dfs.append(df)
df_all = pd.concat(dfs)
df_all.columns = ['qseqid','sseqid','qstart','qend','sstart','send','score','length','mismatch','gaps','gapopen','nident','pident','qlen','slen','qcovs']
df_all = df_all.reset_index(drop=True)
print('initial:',len(df_all.index))

('initial:', 7315)


In [4]:
df_all['qseqid'].head()

0    LTR_id_7512_RLG_singleton_family3_LTR_62_ChrUn_2310158_2311752[type:ltr_source:tephra][type:ltr_source:tephra]
1    LTR_id_7374_RLG_LTR_296_FAM002621[type:ltr_source:TASR][type:ltr_source:TASR]                                 
2    LTR_id_5335_RLG_LTR_606_FAM007493[type:ltr_source:TASR][type:ltr_source:TASR]                                 
3    LTR_id_2672_RLC_LTR_519_FAM006723[type:ltr_source:TASR][type:ltr_source:TASR]                                 
4    LTR_id_7319_RLG_LTR_485_FAM005744[type:ltr_source:TASR][type:ltr_source:TASR]                                 
Name: qseqid, dtype: object

In [5]:
df_all['source'] = 'ST_annotation'
df_all['frame'] = '.'
df_all['score'] = '.'
df_all['strand'] = '+'
df_all['feature'] = 'TE'
splited_id = df_all['qseqid'].str.split('_')
df_all['auto_id'] = splited_id.str[0] + '_' + splited_id.str[1] + ':' + splited_id.str[2] + ';'
df_all['qseqid'] = df_all['qseqid'].str.split('[', 1).str[1] + ';source_id:' + df_all['qseqid'].str.split('[', 1).str[0] 
df_all['qseqid'] = df_all['qseqid'].str.replace("\\]\\[",";")
df_all['qseqid'] = df_all['qseqid'].str.replace("]","")
df_all['qseqid'] = df_all['qseqid'].str.replace("[","")
df_all['qseqid'] = df_all['qseqid'].str.replace("[","")

In [6]:
#order by chromosome and position
df_all['new_sstart'] = df_all[['sstart','send']].min(axis=1)
df_all['new_ssend'] = df_all[['sstart','send']].max(axis=1)
df_all['sstart'] = df_all['new_sstart']
df_all['send'] = df_all['new_ssend']
df_all = df_all.drop('new_sstart',axis=1).drop('new_ssend',axis=1)
df_all.sort_values(['sseqid', 'sstart'], inplace=True)

In [7]:
#delete dups annotation
for k,v in df_all.iterrows():
    qs = {}
    for q in v.qseqid.split(";"):
        qs[q] = 1
    new_qseqid = ";".join(qs.keys())
    df_all.at[k, 'qseqid'] = new_qseqid

In [8]:
df_all['qseqid'] = df_all['auto_id'] + df_all['qseqid']
df_all = df_all[['sseqid','source','feature','sstart','send','score','strand','frame','qseqid']]
#gff3 -> ['seqname' , 'source' , 'feature' , 'start' , 'end' , 'score' , 'strand' , 'frame' , 'attribute']

In [9]:
df_all.to_csv(path_output, sep="\t", header=None, index=None)
path_output

'data/results/all.gff'